# Implementação do Algoritmo Simplex

### Importação de bibliotecas

In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt

### Inicialização do problema

In [2]:
c = [1, 1, 1, 0]  # coefficients for [y₁, y₂, s₁, s₂]

# Constraint matrix
A = [
    [1, 2, 3, 0],  
    [-1, 2, 6, 0],   
    [0, 4, 9, 0],   
    [0, 0, 3, 1]   
]

# RHS values
b = [3, 2, 5, 1]



In [3]:
#----// Funcoes de Print //----------------


def print_tableau(tableau):
    rows = len(tableau)
    cols = len(tableau[0])
    
    header = ['x' + str(i+1) for i in range(cols-1)] + ['RHS']
    print('    ' + ''.join(f'{col:>8}' for col in header))
    print('    ' + '-' * (8 * cols))
    
    for i in range(rows-1):
        print(f'R{i+1}: ' + ''.join(f'{val:8.2f}' if val is not None else '   None ' for val in tableau[i]))
    
    print('z : ' + ''.join(f'{val:8.2f}' if val is not None else '   None ' for val in tableau[-1]))
    print()


def print_matrix(matrix):
    for row in matrix:
        print(''.join(f'{val:8.2f}' for val in row))


#----// Funcoes Basicas de Tableu //----------------

def eh_basica(coluna):
    return sum(coluna) == 1 and len([c for c in coluna if c == 0]) == len(coluna) - 1


def get_one(vector):
    for i in range(len(vector)):
        if vector[i] == 1:
            return i

def get_indices(matrix):
    slots = [0] * len(matrix)
    colunas = np.array(matrix).T
    i_basicos = []
    for coluna in range(0, len(colunas)):
        if eh_basica(colunas[coluna]) and slots[get_one(colunas[coluna])] == 0:
            i_basicos.append(coluna)
            slots[get_one(colunas[coluna])] = 1
            print("slots: ", slots)

    print("i_basicos: ", i_basicos)
    return i_basicos


def get_slots(matrix):
    slots = [0] * len(matrix)
    colunas = np.array(matrix).T
    for coluna in range(0, len(colunas)):
        if eh_basica(colunas[coluna]) and slots[get_one(colunas[coluna])] == 0:
            slots[get_one(colunas[coluna])] = 1

    return slots

def identify_basic_variables(A):
    A = np.array(A)
    basic_vars = []
    cols = A.T
    for j, col in enumerate(cols):
        if eh_basica(col):
            basic_vars.append(j)
    return basic_vars

#----// Faze 1 //----------------


def fase1(c, A, b):
    #Checagem se a solução inicial tem solucao indentidade LI
    if len(A) == len(get_indices(A)):
        return c, A, b
 
 
    # Se nessesario multiplicar a linha por -1 para que o RHS seja positivo
    for line in range(0, len(b)):
        if b[line] < 0:
            A[line] = -A[line]
            b[line] = -b[line]   


    c = [0] * len(c)
    slots = get_slots(A)
    for i in range(0, len(slots)):
        if slots[i] == 0:
            coluna = [0] * len(slots)
            coluna[i] = 1
            for linha in range(0, len(A)):
                A[linha].append(coluna[linha])
            c.append(1)

    print("+++++++++++++++++++++++++++")
    print_matrix(A)
    #indices = get_indices(A)
    print("C:", c)
    print("+++++++++++++++++++++++++++")




    return c, A, b




#----// Pre-Simplex //----------------


def calcular_z(c, basic_vars, b):
    z = sum(c[j] * b_i for j, b_i in zip(basic_vars, b))
    return z


def para_tableau(c, A, b):
    # Create the main part of the tableau
    xb = []
    for equacao, termo_independente in zip(A, b):
        linha_tableau = equacao + [termo_independente]
        xb.append(linha_tableau)
    
    # Identify basic variables
    basic_vars = identify_basic_variables(A)
    
    # Calculate z-value based on basic variables
    z = calcular_z(c, basic_vars, b)
    
    # Construct z-row: negative of objective coefficients
    z_row = [-coef for coef in c] + [z]
    
    return xb + [z_row]


#----// Simplex //----------------



def pode_melhorar(tableau):
    #Verifica se a solução é ótima. Se algum coeficiente na linha de custo (função objetivo) é negativo, ainda não é ótima.
    
    return any(x < 0 for x in tableau[-1][:-1])


def obter_posicao_pivo(tableau):
    z = tableau[-1]
    coluna = next(i for i, x in enumerate(z[:-1]) if x < 0)  # Changed from > to <
    
    restricoes = []
    for eq in tableau[:-1]:
        el = eq[coluna]
        restricoes.append(math.inf if el <= 0 else eq[-1] / el)

    linha = restricoes.index(min(restricoes))
    return linha, coluna


def passo_pivo(tableau, posicao_pivo):
    novo_tableau = [[] for eq in tableau]
    
    i, j = posicao_pivo
    valor_pivo = tableau[i][j]
    novo_tableau[i] = np.array(tableau[i]) / valor_pivo
    
    for eq_i, eq in enumerate(tableau):
        if eq_i != i:
            multiplicador = np.array(novo_tableau[i]) * tableau[eq_i][j]
            novo_tableau[eq_i] = np.array(tableau[eq_i]) - multiplicador
   
    return novo_tableau



def obter_solucao(tableau):
    colunas = np.array(tableau).T
    solucoes = []
    for coluna in colunas:
        solucao = 0
        if eh_basica(coluna):
            print("Coluna Basica:", coluna)
            coluna_lista = coluna.tolist()
            indice_um = coluna_lista.index(1)   # Encontrando o indice onde aparece o valor 1 na coluna e pegando o RHS
            solucao = colunas[-1][indice_um]
        solucoes.append(solucao)
        
    return solucoes

def simplex(c, A, b):
    tableau = para_tableau(c, A, b)
    print_tableau(tableau)
    while pode_melhorar(tableau):
        posicao_pivo = obter_posicao_pivo(tableau)
        tableau = passo_pivo(tableau, posicao_pivo)
        print_tableau(tableau)

    return obter_solucao(tableau)






In [4]:

#------------------------------



c, A, b = fase1(c, A, b)

#i_basicos = get_indices(A)

solucao = simplex(c, A, b)
print(solucao)


slots:  [0, 0, 0, 1]
i_basicos:  [3]
+++++++++++++++++++++++++++
    1.00    2.00    3.00    0.00    1.00    0.00    0.00
   -1.00    2.00    6.00    0.00    0.00    1.00    0.00
    0.00    4.00    9.00    0.00    0.00    0.00    1.00
    0.00    0.00    3.00    1.00    0.00    0.00    0.00
C: [0, 0, 0, 0, 1, 1, 1]
+++++++++++++++++++++++++++
          x1      x2      x3      x4      x5      x6      x7     RHS
    ----------------------------------------------------------------
R1:     1.00    2.00    3.00    0.00    1.00    0.00    0.00    3.00
R2:    -1.00    2.00    6.00    0.00    0.00    1.00    0.00    2.00
R3:     0.00    4.00    9.00    0.00    0.00    0.00    1.00    5.00
R4:     0.00    0.00    3.00    1.00    0.00    0.00    0.00    1.00
z :     0.00    0.00    0.00    0.00   -1.00   -1.00   -1.00    8.00

          x1      x2      x3      x4      x5      x6      x7     RHS
    ----------------------------------------------------------------
R1:     1.00    2.00    3.00    

In [5]:
# Objective function coefficients (c)
c = [-2, -1, 0, 0]  # coefficients for [x₁, x₂, s₁, s₂]

# Constraint coefficients matrix (A)
A = [
    [1, -1, 1, 0],  # x₁ - x₂ + s₁ = 1
    [1,  2, 0, 1]   # x₁ + 2x₂ + s₂ = 7
]

# Right-hand side values (b)
b = [1, 7]


solucao = simplex(c, A, b)
print('solução: ', solucao)

          x1      x2      x3      x4     RHS
    ----------------------------------------
R1:     1.00   -1.00    1.00    0.00    1.00
R2:     1.00    2.00    0.00    1.00    7.00
z :     2.00    1.00    0.00    0.00    0.00

Coluna Basica: [1 0 0]
Coluna Basica: [0 1 0]
solução:  [0, 0, np.int64(1), np.int64(7), 0]


## Problema Pratico

In [6]:
solucao = simplex(c, A, b)
print('solução do problema prático: ', solucao)

          x1      x2      x3      x4     RHS
    ----------------------------------------
R1:     1.00   -1.00    1.00    0.00    1.00
R2:     1.00    2.00    0.00    1.00    7.00
z :     2.00    1.00    0.00    0.00    0.00

Coluna Basica: [1 0 0]
Coluna Basica: [0 1 0]
solução do problema prático:  [0, 0, np.int64(1), np.int64(7), 0]



    Implicit Initial Basic Solution
    The initial basic feasible solution is:

    Set all original variables (x₁, x₂) to 0
    Slack variables (x₃, x₄, x₅) take the values of the RHS (b vector)
    This works because the slack variables form an identity matrix in the constraint matrix

Important Note: This code assumes the problem is already in standard form and doesn't implement Phase I of the simplex method. It would need modification to handle:

    Problems with ≥ constraints
    Problems with negative RHS values
    Problems requiring artificial variables

To make this code more robust, you would need to add Phase I implementation to handle cases where an initial basic feasible solution isn't immediately available.


In [7]:
# Objective function coefficients
# For minimization: Min 4x₁ + 3x₂ + 7x₃
c = [4, 3, 7, 0, 0, 0]  # Negative for minimization

# Constraint matrix with slack variables
A = [
    [2, 2, 1, 0, 0, 0],  # Added slack variables [1, 0] for first constraint
    [3, 1, 2, 0, 0, 0],  # Added slack variables [0, 1] for second constraint
    [1, 0, 0, 1, 0, 0],  # x₁ ≥ 0 
    [0, 1, 0, 0, 1, 0],  # x₂ ≥ 0
    [0, 0, 1, 0, 0, 1]   # x₃ ≥ 0
]

# RHS values including non-negativity constraints
b = [12, 8, 0, 0, 0]


solucao = simplex(c, A, b)
print('solução do problema prático: ', solucao)

          x1      x2      x3      x4      x5      x6     RHS
    --------------------------------------------------------
R1:     2.00    2.00    1.00    0.00    0.00    0.00   12.00
R2:     3.00    1.00    2.00    0.00    0.00    0.00    8.00
R3:     1.00    0.00    0.00    1.00    0.00    0.00    0.00
R4:     0.00    1.00    0.00    0.00    1.00    0.00    0.00
R5:     0.00    0.00    1.00    0.00    0.00    1.00    0.00
z :    -4.00   -3.00   -7.00    0.00    0.00    0.00    0.00

          x1      x2      x3      x4      x5      x6     RHS
    --------------------------------------------------------
R1:     0.00    2.00    1.00   -2.00    0.00    0.00   12.00
R2:     0.00    1.00    2.00   -3.00    0.00    0.00    8.00
R3:     1.00    0.00    0.00    1.00    0.00    0.00    0.00
R4:     0.00    1.00    0.00    0.00    1.00    0.00    0.00
R5:     0.00    0.00    1.00    0.00    0.00    1.00    0.00
z :     0.00   -3.00   -7.00    4.00    0.00    0.00    0.00

          x1      x2  

In [9]:

# Objective function coefficients (minimization, so we use negative values)
c = [4, 3, 7]  # coefficients for [y₁, y₂, s₁, s₂]

# Constraint matrix
A = [
    [2, 2, 1],  
    [3, 1, 2]
]

# RHS values
b = [12, 8]

solucao = simplex(c, A, b)
print('solução do problema prático: ', solucao)

          x1      x2      x3     RHS
    --------------------------------
R1:     2.00    2.00    1.00   12.00
R2:     3.00    1.00    2.00    8.00
z :    -4.00   -3.00   -7.00    0.00

          x1      x2      x3     RHS
    --------------------------------
R1:     0.00    1.33   -0.33    6.67
R2:     1.00    0.33    0.67    2.67
z :     0.00   -1.67   -4.33   10.67

          x1      x2      x3     RHS
    --------------------------------
R1:     0.00    1.00   -0.25    5.00
R2:     1.00    0.00    0.75    1.00
z :     0.00    0.00   -4.75   19.00

          x1      x2      x3     RHS
    --------------------------------
R1:     0.33    1.00    0.00    5.33
R2:     1.33    0.00    1.00    1.33
z :     6.33    0.00    0.00   25.33

Coluna Basica: [1. 0. 0.]
Coluna Basica: [0. 1. 0.]
solução do problema prático:  [0, np.float64(5.333333333333333), np.float64(1.3333333333333335), 0]
